## Exercise 11: MINOS Neutrino Experiment

In [1]:
import ROOT

In [2]:
# declare observable E (GeV) the reconstructed neutrino energy from range 0.5 to 14 GeV
E = ROOT.RooRealVar("E", "Neutrino Energy [GeV]", 0.5, 14)

# Load the unbinned dataset of observed events into a RooDataSet
data = ROOT.RooDataSet.read("minos_2013_data.dat", ROOT.RooArgList(E), "v")

[#1] INFO:DataHandling -- RooDataSet::read: reading file minos_2013_data.dat
[#1] INFO:DataHandling -- RooDataSet::read: read 3470 events (ignored 0 out of range events)


In [3]:
# visualize the data
c = ROOT.TCanvas()
frame = E.frame()
frame.SetTitle("Neutrino Energy Spectrum")
data.plotOn(frame)
frame.Draw()
c.Draw()

#### PART 1 - Build the model

Create a PDF describing the energy spectrum and how the spectrum is altered by neutrino oscillations.

In [4]:
# Load the non-oscillated Monte-Carlo sample
mc_no_osc = ROOT.RooDataSet.read("minos_2013_mc.dat", ROOT.RooArgList(E), "v")

[#1] INFO:DataHandling -- RooDataSet::read: reading file minos_2013_mc.dat
[#1] INFO:DataHandling -- RooDataSet::read: read 48295 events (ignored 0 out of range events)


In [5]:
# Turn this MC sample into a histogram-based function
# This function represents the shape of the energy distribution for non-oscillated neutrinos

# reduce the dataset to observable E
dd = mc_no_osc.reduce(ROOT.RooArgSet(E))
# create a binned dataset using RooDataHist
dh_mc_no_osc = ROOT.RooDataHist("dh_mc_no_osc", "Binned MC No Oscillation", ROOT.RooArgList(E), dd)
# create a RooHistFunc object from the RooDataHist
func_no_osc = ROOT.RooHistFunc("func_no_osc", "Energy Spectrum of non-oscillating Neutrinos", ROOT.RooArgList(E), dh_mc_no_osc, 2)

$𝑃(\nu_\mu \rightarrow \nu_\mu) = 1 − \sin^2(2\theta) \sin^2(1.267 \Delta m^2 \frac{L}{E}) $ [Eq. 1]  
 
𝐿 = 730 km

In [6]:
### Construct the oscillation probability function ###

# create parameters
mixing = ROOT.RooRealVar("mixing", "\\sin^{2}(2\\theta)", 0.9, 0.0, 1.0)
dm2 = ROOT.RooRealVar("dm2", "\\Delta m^{2} [eV^{2}]", 0.0024, 0.0001, 0.01)
L = ROOT.RooRealVar("L", "Baseline", 730)   # constant

osc_prob = ROOT.RooFormulaVar("osc_prob", "Oscillation Probability",
                            "1 - mixing * pow(TMath::Sin(1.267*dm2*L/E), 2)", ROOT.RooArgList(mixing, dm2, L, E))

In [7]:
### Build the final model ###
# The shape of the energy distribution for oscillated neutrinos is obtained 
# by modulating the non-oscillated spectrum function by the oscillation probability

model = ROOT.RooGenericPdf("model", "model", "@0 * @1", ROOT.RooArgSet(osc_prob, func_no_osc))

In [8]:
### Fit and plot ### 
# Fit the model to the data and save the plot as minos_data.png
model.fitTo(data)

c2 = ROOT.TCanvas("c2", "c2")
frame2 = E.frame()
frame2.SetTitle("Fitted Model to MINOS Data")

data.plotOn(frame2, ROOT.RooFit.Name("Data"))
model.plotOn(frame2, ROOT.RooFit.Name("Fitted Model"))

frame2.Draw()

legend = ROOT.TLegend(0.5, 0.7, 0.8, 0.8)
legend.AddEntry(frame2.findObject("Data"), "MINOS Data", "lep")
legend.AddEntry(frame2.findObject("Fitted Model"), "Best Fit Oscillations", "l")

legend.Draw()

c2.Draw()
c2.SaveAs("minos_data.png")

[#1] INFO:Fitting -- RooAbsPdf::fitTo(model_over_model_Int[E]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_over_model_Int[E]_dataset) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
Minuit2Minimizer: Minimize with max-calls 1000 convergence for edm < 1 strategy 1
[#1] INFO:NumericIntegration -- RooRealIntegral::init(model_Int[E]) using numeric integrator RooIntegrator1D to calculate Int(E)
Minuit2Minimizer : Valid minimum - status = 0
FVAL  = 8396.08624553286609
Edm   = 5.38082540795448178e-07
Nfcn  = 31
dm2	  = 0.00232974	 +/-  7.31696e-05	(limited)
mixing	  = 0.885095	 +/-  0.0253974	(limited)
[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: deactivating const optimization
[#1] INFO:NumericIntegrati

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       8397.532516 Edm =       2.232505425 NCalls =      9
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 8397.532516
  Edm           : 2.232505425
  Internal parameters:	[    -0.5649262609      0.927295218]	
  Internal gradient  :	[      112.9547969      22.18851147]	
  Internal covariance matrix:
[[  0.00038897856              0]
 [              0   0.0080578619]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1000
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       8397.532516 Edm =       2.232505425 NCalls =      9
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       8396.098298 Edm =    0.004961453073 NCalls =     15
Info in <Minuit2>: VariableMetricBuilder    2 - FCN =       8396.086246 Edm =   5.155714296e-07 NCalls =     21
Info in <Minuit2>: VariableMetr

#### PART 2 - Likelihood Minimization with RooMinimizer

In [9]:
### Minimize the Likelihood ###
# Construct the unbinned negative log-likelihood (NLL) of the model with respect to the generated dataset
nll = model.createNLL(data)

# Create a MINUIT interface. Pass the nll object to the constructor
minuit = ROOT.RooMinimizer(nll)
minuit.setVerbose(True)

[#1] INFO:Fitting -- RooAbsPdf::fitTo(model_over_model_Int[E]) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_model_over_model_Int[E]_dataset) Summation contains a RooNLLVar, using its error level


In [10]:
# Run MIGRAD to perform likelihood minimization
minuit.migrad()

0

Minuit2Minimizer: Minimize with max-calls 1000 convergence for edm < 1 strategy 1
[#1] INFO:NumericIntegration -- RooRealIntegral::init(model_Int[E]) using numeric integrator RooIntegrator1D to calculate Int(E)

prevFCN = 8396.086246  dm2=0.002334, 
prevFCN = 8396.089162  dm2=0.002325, 
prevFCN = 8396.089012  dm2=0.00233, mixing=0.8867, 
prevFCN = 8396.08918  mixing=0.8835, 
prevFCN = 8396.089047  dm2=0.00233, mixing=0.8851, 
prevFCN = 8396.086245  dm2=0.00233, mixing=0.8851, 
prevFCN = 8396.086245  dm2=0.002334, 
prevFCN = 8396.088251  dm2=0.002326, 
prevFCN = 8396.088243  dm2=0.00233, mixing=0.8864, 
prevFCN = 8396.088246  mixing=0.8837, 
prevFCN = 8396.088247  mixing=0.8851, 
prevFCN = 8396.086245  dm2=0.002334, 
prevFCN = 8396.088251  dm2=0.002326, 
prevFCN = 8396.088243  dm2=0.00233, mixing=0.8864, 
prevFCN = 8396.088246  mixing=0.8837, 
prevFCN = 8396.088247  dm2=0.00233, mixing=0.8851, 
prevFCN = 8396.086325  dm2=0.002329, 
prevFCN = 8396.086325  dm2=0.00233, 
prevFCN = 8396.086

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       8396.086246 Edm =   8.842224974e-07 NCalls =      5
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 8396.086246
  Edm           : 8.842224974e-07
  Internal parameters:	[    -0.5818235463     0.8791391149]	
  Internal gradient  :	[    0.06728242742    0.01321705522]	
  Internal covariance matrix:
[[  0.00043809221              0]
 [              0    0.008893895]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.001 with call limit = 1000
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       8396.086246 Edm =   8.842224974e-07 NCalls =      5
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       8396.086245 Edm =   1.908735171e-09 NCalls =     11
Info in <Minuit2>: VariableMetricBuilder After Hessian
Info in <Minuit2>: VariableMetricBuilder    2 - FCN =       8396.086245 Edm =   8.19

In [11]:
# Print the values of the parameters mixing, dm2
# These values now reflect the MIGRAD optimization
mixing.Print()
dm2.Print()

RooRealVar::mixing = 0.8851 +/- 0.0254  L(0 - 1) 
RooRealVar::dm2 = 0.00233 +/- 7.317e-05  L(0.0001 - 0.01) 


In [12]:
### HESSE Error Calculation ###
# disable verbose mode
minuit.setVerbose(False)
# run HESSE error calculation which computes errors using the second derivatives of the NLL at the minimum
minuit.hesse()

0

Info in <Minuit2>: Minuit2Minimizer::Hesse Using max-calls 1000
Info in <Minuit2>: Minuit2Minimizer::Hesse Hesse is valid - matrix is accurate


In [13]:
# Print the updated parameter values
# These now contain the HESSE error estimates
mixing.Print()
dm2.Print()

RooRealVar::mixing = 0.8851 +/- 0.02529  L(0 - 1) 
RooRealVar::dm2 = 0.00233 +/- 7.286e-05  L(0.0001 - 0.01) 


We notice that the errors have changed a little.

In [14]:
### MINOS Error Calculation for dm2 ###
# Run MINOS for parameter dm2
minuit.minos(dm2)

0

******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS LOWER error for parameter #0 : dm2 using max-calls 1000, tolerance 1
******************************************************************************************************
Minuit2Minimizer::GetMinosError - Run MINOS UPPER error for parameter #0 : dm2 using max-calls 1000, tolerance 1
Minos: Lower error for parameter dm2  :  -7.17e-05
Minos: Upper error for parameter dm2  :  7.408e-05


Info in <Minuit2>: MnMinos Determination of lower Minos error for parameter 0
Info in <Minuit2>: MnFunctionCross Run Migrad with fixed parameters:
  Pos 0: dm2 = 0.00225683
Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       8396.603019 Edm =    6.77769857e-05 NCalls =      3
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 8396.603019
  Edm           : 6.77769857e-05
  Internal parameters:	[     0.9221779283]	
  Internal gradient  :	[     0.1745883596]	
  Internal covariance matrix:
[[   0.0088942979]]]
Info in <Minuit2>: VariableMetricBuilder Start iterating until Edm is < 0.0005 with call limit = 1000
Info in <Minuit2>: VariableMetricBuilder    0 - FCN =       8396.603019 Edm =    6.77769857e-05 NCalls =      3
Info in <Minuit2>: VariableMetricBuilder    1 - FCN =       8396.602945 Edm =   1.181931845e-10 NCalls =      7
Info in <Minuit2>: MnFunctionCross Result after M

In [15]:
# Print the parameter values again.
# Note that the uncertainties for dm2 are now asymmetric
mixing.Print()
dm2.Print()

RooRealVar::mixing = 0.8851 +/- 0.02529  L(0 - 1) 
RooRealVar::dm2 = 0.00233 +/- (-7.17e-05,7.408e-05)  L(0.0001 - 0.01) 


In [16]:
### Save Results ###
fit_results = minuit.save()
# Print the result in verbose mode
fit_results.Print("v")


  RooFitResult: minimized FCN value: 8396, estimated distance to minimum: 8.083e-10
                covariance matrix quality: Full, accurate covariance matrix
                Status : MIGRAD=0 HESSE=0 MINOS=0 

    Constant Parameter    Value     
  --------------------  ------------
                     L    7.3000e+02

    Floating Parameter  InitialValue    FinalValue (+HiError,-LoError)    GblCorr.
  --------------------  ------------  ----------------------------------  --------
                   dm2    2.3297e-03    2.3297e-03 (+7.41e-05,-7.17e-05)  <none>
                mixing    8.8510e-01    8.8508e-01         +/-  2.53e-02  <none>



In [17]:
### Contour Plot ###
# Produce a contour plot of dm2 vs mixing corresponding to the confidence levels of 68%, 95.45%, 99.73%
frame = minuit.contour(mixing, dm2, 1, 2, 3)
frame.SetTitle("Contour Plot")

c = ROOT.TCanvas()
frame.GetYaxis().SetRangeUser(1.9e-3, 3.5e-3)
frame.GetXaxis().SetRangeUser(0.8, 0.95)
frame.Draw()
c.Draw()

[#0] ERROR:Minimization -- RooMinimizer::contour(RooMinimizer) ERROR: MINUIT did not return a contour graph for n=3


Info in <Minuit2>: Minuit2Minimizer::Contour Computing contours - 0.5
Info in <Minuit2>: Minuit2Minimizer::Contour Computing contours - 2
Info in <Minuit2>: Minuit2Minimizer::Contour Computing contours - 4.5
Error in <Minuit2>: MnContours unable to find Lower y Value for x Parameter 1
Error in <Minuit2>: Minuit2Minimizer::Contour Invalid result from MnContours
